In [ ]:
import sys
sys.path.append('..')

from utils import show
import image_processes as imgp
import map_processes as mapp
import utils as ut
import numpy as np
import cv2
import os
import numpy as np
from skimage.morphology import medial_axis
from skimage.util import invert
from skimage.io import imread
import json

%load_ext autoreload
%autoreload 2


In [ ]:
imgName = 'map.jpg'
projectName = 'map4096'
texturePath = '../../../textures/map4096/'
outputPath = './outputs/'
outputPath = os.path.join(outputPath, projectName)

imgPath = os.path.join(texturePath, imgName)
# gray to get the height and width with shape
image = imread(imgPath, as_gray=True)
height, width = image.shape

# imgSize = (int(width/8), int(height/8))
imgSize = (width, height)
image = cv2.resize(image, imgSize)

if not os.path.exists(outputPath):
    os.makedirs(outputPath)

inputPath = os.path.join(outputPath, 'input.png')
cv2.imwrite(inputPath, cv2.cvtColor(imread(imgPath), cv2.COLOR_BGR2RGB))


def getPath(filename):
    return os.path.join(outputPath, filename)

In [ ]:
image = cv2.imread(imgPath)
image = cv2.resize(image, imgSize)
imgp.convertToRGB(image, False);

In [ ]:
gray = imgp.convertToGray(image, False)

In [ ]:
# Threshold
grayThreshold = 120
thresholded = imgp.threshold(gray, grayThreshold, True)

In [ ]:
# Skeleton
sensitivity = 0.47
gray_normalized = imgp.imreadGrayNormalized(imgPath, imgSize)
skeleton = imgp.skeletonize(gray_normalized, sensitivity, True)
ut.saveBinaryImage(getPath("skeleton.png"), skeleton);

In [ ]:
# Get contours
contours, points = imgp.getContours(gray, grayThreshold)
print(len(contours))

# Draw contours
filled_image = imgp.drawContours(skeleton, contours, False)

# Test filling
imgp.fillContours(skeleton, filled_image, points, True)

In [ ]:
provinceDataViews, mapColors, mapDebug = mapp.doMapQuantization(
    skeleton, points, True, True)

cv2.imwrite(getPath("map_colors.png"),
            cv2.cvtColor(mapColors, cv2.COLOR_BGR2RGB))

if mapDebug is not None:    
    cv2.imwrite(getPath("map_colors_debug.png"),
                cv2.cvtColor(mapDebug, cv2.COLOR_BGR2RGB))

In [ ]:
# CREATING JSON
data = mapp.createMapJson(
    projectName, 10, provinceDataViews, width+1024, height+1024)
ut.writeJson(data, outputPath, projectName)

In [ ]:
# CREATING BORDERS
invertedSkeleton = invert(skeleton)
color = imgp.hex_to_rgba("#614338")
mapBorders = imgp.thickening(
    invertedSkeleton, 8, 4, 80, color, True, False)
ut.saveCVImage(outputPath, 'map_borders.png',
               mapBorders)


In [ ]:
# Generate glow map
glowInput = cv2.cvtColor(mapColors, cv2.COLOR_RGB2RGBA)
mapGlow = imgp.alphaSubtraction(glowInput, mapBorders, True)
ut.saveCVImage(outputPath, 'map_glow.png', mapGlow)

In [ ]:
# Generate debug map
mask = imgp.createBlankImage(width, height, (60, 20, 90, 255))
ut.saveCVImage(outputPath, 'map_debug.png', mask)